# Initial setup
- Library imports
- Utility functions

In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from glob import glob
import os
from joblib import Parallel, delayed

from PIL import Image
import requests
from io import BytesIO

import torch
import torch.nn as nn 
from torch.utils.data import Dataset

In [2]:
LANES = ["TOP", "JUNGLE", "MIDDLE", "BOTTOM", "UTILITY"]

with open(os.path.join("game_data", "items.json"), "r", encoding="utf-8") as fp:
    item_dict = json.load(fp)
    ITEMS = {
        item_id: item_info
        for item_id, item_info in item_dict.items()
        if item_info["tier"] >= 3 or (item_info["tier"] == 2 and \
            ("BOOTS" in item_info["rank"] or item_info["id"] in [3004, 3003, 3119, 3010, 3866])) or \
            ("STARTER" in item_info["rank"])
    }

with open(os.path.join("game_data", "champions.json"), "r", encoding="utf-8") as fp:
    CHAMP_DICT = json.load(fp)
    CHAMP_NAME_TO_ID = {c: CHAMP_DICT[c]["id"] for c in CHAMP_DICT.keys()}
    CHAMP_ID_TO_NAME = {CHAMP_DICT[c]["id"]: c for c in CHAMP_DICT.keys()}
    CHAMPION_ICONS = {c: CHAMP_DICT[c]["icon"] for c in CHAMP_DICT.keys()}

with open(os.path.join("game_data", "perks.json"), "r", encoding="utf-8") as fp:
    PERK_ID_TO_NAME = json.load(fp)

def get_item_name(item_id: int):
    return ITEMS.get(str(int(item_id)), "").get("name")

def get_item_icon(item_id: int):
    if not item_id:
        return None
    url = ITEMS.get(str(int(item_id)), {}).get("icon", "https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/data/spells/icons2d/summoner_empty.png")
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        return img
    except:
        return Image.new("RGB", (64, 64))

def get_champion_id(champion_name: str):
    if champion_name is None:
        return None
    return CHAMP_NAME_TO_ID.get(champion_name, "")

def get_champion_name(champion_id: int):
    return CHAMP_ID_TO_NAME.get(champion_id)

def get_champion_icon(champion_name: int):
    if not champion_name:
        return None
    url = CHAMPION_ICONS.get(champion_name)
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        return img
    except:
        return Image.new("RGB", (64, 64))

# Base model
A model that predicts 6 completely random item id's and compares them to the real items.

It's used as a base model for the models trained in `full_build_prediction.ipynb`.

In [ ]:
list_info = []
max_size = 1000000
size = 0
for file in glob("features/postgame/*.json"):
    with open(file, "r", encoding="UTF-8") as fp:
        info = json.load(fp)
    for matchId in info.keys():
        for participant_info in info[matchId]:
            items = participant_info["items"]
            list_info.append(items + [0] * (6 - len(items)))
            size += 1
    if size >= max_size:
        break
df = pd.DataFrame(list_info, columns=[f"item_{i}" for i in range(6)])

In [12]:
df.head()

,item_0,item_1,item_2,item_3,item_4,item_5
0,3071,1055,6610,3047,3161,0
1,2504,3143,3047,3110,0,0
2,3107,3222,1082,6617,6621,3158
3,3100,3152,3175,0,0,0
4,3152,3157,3020,0,0,0


In [11]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_score, recall_score

np.random.seed(42)

# All item ids
item_ids = list([int(id) for id in ITEMS.keys()])
n_samples = len(df)
# For each row, 6 predictions sampled from item_ids
pred_data = [
    list(np.random.choice(item_ids, size=6, replace=False))
    for _ in range(n_samples)
]
# Learn classes from the dataset + predictions
all_labels = df.values.tolist() + pred_data
mlb = MultiLabelBinarizer()
mlb.fit(all_labels)
# Transform both datasets
y_test_bin = mlb.transform(df.values.tolist())
pred_bin = mlb.transform(pred_data)
# Find the column for item 0 and remove it (if present)
if 0 in mlb.classes_:
    zero_index = list(mlb.classes_).index(0)
    y_test_bin = np.delete(y_test_bin, zero_index, axis=1)
    pred_bin = np.delete(pred_bin, zero_index, axis=1)
# Compute metrics
recall_at_6 = recall_score(y_test_bin, pred_bin, average='samples', zero_division=0)
precision_at_6 = precision_score(y_test_bin, pred_bin, average='samples', zero_division=0)
print(f"Recall@6: {recall_at_6:.8f}")
print(f"Precision@6: {precision_at_6:.8f}")

Recall@6: 0.04405598
Precision@6: 0.02858198
